In [ ]:
# TRAIN THE MODEL ON CUSTOM DATASET

from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# fliplr=0.0, mosaic=0.0, degrees=30.0, close_mosaic=0,
# Use the model
model.train(data="football_number.yaml", epochs=60, batch=32, mosaic=0.0, cos_lr=True, name="numberDetector")  # train the model
metrics = model.val()  # evaluate model performance on the validation set

In [ ]:
# TEST THE MODEL ON A SINGLE IMAGE AND PLOT THE RESULT


%matplotlib inline

import torch
from torchvision.transforms import Grayscale, InterpolationMode
from ultralytics import YOLO
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torchvision.transforms.functional import resize
import numpy as np
import cv2


MODEL_PATH = 'runs/detect/v4/weights/last.pt'
model = YOLO(MODEL_PATH)
# model.eval()


mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

TEST_IMAGE_PATH = 'SoccerNet/jersey-2023/train/images/0/0_3.jpg'

img= read_image(TEST_IMAGE_PATH)
img = resize(img, (640, 640)) / 255.0
#img = Grayscale(num_output_channels=3)(img)
#img = (img - mean) / std

detected_numberBox = model.predict(img.unsqueeze(0), verbose=False)
print(detected_numberBox[0].boxes.xyxy)

x1, y1, x2, y2 = 0, 0, 0, 0

img_numpy = img.permute(1, 2, 0).numpy()
image = img_numpy.copy()
#Draw bounding box on the image
for box in detected_numberBox[0].boxes.xyxy:
    x1, y1, x2, y2 = box.tolist()
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    

cropped_image = img_numpy[y1:y2, x1:x2]
plt.imshow(image)
plt.axis('off')  # Rimuovi gli assi
plt.savefig("result.png", bbox_inches='tight', pad_inches=0)

In [ ]:
# TEST THE MODEL OVER A n NUMBER OF SAMPLES
# Results with bounding boxe traced on the image saved in the folder "detections"


import random
import os
from pathlib import Path

%matplotlib inline

import torch
from torchvision.transforms import Grayscale, InterpolationMode
from ultralytics import YOLO
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torchvision.transforms.functional import resize
import numpy as np
import cv2
import os
from torchvision.transforms import v2


destination_folder = Path(os.path.join('detections/'))
destination_folder.mkdir(parents=True, exist_ok=True)

model = YOLO('runs/detect/train12/weights/last.pt')
# model.eval()

InterpolationMode.BILINEAR
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

train_images = []
for id in os.listdir('SoccerNet/jersey-2023/train/images'):
    if id != '.DS_Store':
        for image in os.listdir('SoccerNet/jersey-2023/train/images/'+id):
            train_images.append(image)

SAMPLES = 50
random.shuffle(train_images)
train_images = train_images[:SAMPLES]

for image_name in train_images:
    # Extract the id from the string
    id = image_name.split('_')[0]

    # Compose the complete path
    img_path = os.path.join('SoccerNet/jersey-2023/train/images', id, image_name)
    img = read_image(img_path)
    img = resize(img, (640, 640)) / 255.0
    #img = v2.Normalize(mean, std)(img)
    img = Grayscale(num_output_channels=3)(img)
    #img = (img - mean) / std

    detected_numberBox = model.predict(img.unsqueeze(0), verbose=False)
    #print(detected_numberBox[0].boxes.xyxy)

    x1, y1, x2, y2 = 0, 0, 0, 0

    img_numpy = img.permute(1, 2, 0).numpy()
    image = img_numpy.copy()
    #Draw bounding box on the image
    for idx, box in enumerate(detected_numberBox[0].boxes.xyxy):
        #print(detected_numberBox[0].boxes.conf[idx])
        #if detected_numberBox[0].boxes.conf[idx] > 0.5:     
            x1, y1, x2, y2 = box.tolist()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            image = cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

    #cropped_image = img_numpy[y1:y2, x1:x2]
    plt.imshow(image)
    plt.axis('off')  # Rimuovi gli assi
    plt.savefig(f"detections/{image_name}", bbox_inches='tight', pad_inches=0)
